## verzeichnisse_extract_data.ipynb

In this notebook, we will extract the text data enclosed in the Verzeichnisse. The data has been pre-OCR'ed, with the layout-ordered text present in the pdf. We will use an algorithm to extract this layout-specific text to order the data in rows and column. 

In [1]:
import re
import fitz
import ollama
import pdfplumber
from fpdf import FPDF

# OpenAI
from openai import OpenAI
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

ollama.pull("llama3.1")
ollama.pull("philsaysss/german-language")

# Function to crop the .pdf
def crop_pdf(pdf_path, output_path, x1=0, x2=180):
    doc = fitz.open(pdf_path)
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        rect = page.mediabox
        crop_rect = fitz.Rect(x1, rect.y0, x2, rect.y1)
        page.set_cropbox(crop_rect)
        
    doc.save(output_path)
    doc.close()    
    
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""
        
        # Loop through all the pages in the PDF
        for page in pdf.pages:
            # Extract text from each page
            text = page.extract_text()
            if text:
                all_text += text
        
    return all_text

def split_text_by_pattern(text):
    # Regex pattern to match a digit followed by a space
    pattern = r'\n\d+\s'
    
    # Split the text using the regex pattern
    split_list = re.split(pattern, text)
    
    return split_list

def crop_pdf_and_extract_text(pdf_path, output_path, x1=0, y1=0, x2=180, y2=722):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Initialize the text container
        extracted_text = ""
        
        # Create a new PDF with fpdf to store cropped text
        pdf_writer = FPDF()
        
        # Iterate over all the pages in the PDF
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            
            # Define the cropping box (x1, y1, x2, y2)
            crop_box = (x1, y1, x2, y2)
            
            # Crop the page to the defined box
            cropped_page = page.within_bbox(crop_box)
            
            # Extract the text from the cropped area
            cropped_text = cropped_page.extract_text()
            
            # Add the cropped text to the total extracted text
            if cropped_text:
                cropped_text = cropped_text.encode('latin-1', 'replace').decode('latin-1')
                extracted_text += cropped_text + "\n"
                
                # Add a new page to the new PDF document
                pdf_writer.add_page()
                pdf_writer.set_xy(10, 10)
                pdf_writer.set_font("Arial", size=12)
                pdf_writer.multi_cell(0, 10, cropped_text)
        
        # Save the new PDF with cropped text
        pdf_writer.output(output_path)

        # Optionally, return the extracted text
        return extracted_text

def request_city_llm(extracted_text):
    responses = []
    for text in extracted_text:
        prompt = f"""
        Welche Stadt wird in diesem Text genannt - der Städtename kann durch einige Füllwörter getrennt werden. 
        Gib mir nur den Städtename, nichts anderes: {text}
        """
        response = ollama.chat(model='philsaysss/german-language', messages=[
            {
                'role': 'user',
                'content': prompt,
                },
            ],
                               options={'temperature':0})
        
        output = response['message']['content']
        responses.append(output)
    return responses


def request_city_gpt(extracted_text):
    responses = []
    for text in extracted_text:
        prompt = f"""
        Welche Stadt wird in diesem Text genannt? Der Städtename kann durch einige Füllwörter getrennt sein. Gib mir nur den Städtenamen, nichts anderes: {text}
        """
        message = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
        ]
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=message,
        max_tokens=2500
        )
        responses.append(response)
        
    return responses
        

## First Attempt

- The first pipeline uses the above two functions to extract text from pdf and then split it by pattern. 

In [18]:
# Specify the path to the PDF file
pdf_file_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/test.pdf'

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_path)

# Output the extracted text
out = split_text_by_pattern(extracted_text)


['A. Verliehene Privilegien .\nFortlaufende Vor- und Zuname, Charakter Gegenstand Quaint Datum D da eu se r Ob die\nGeheimhaltung\nBahl und des Privild ee gr i ums- P gr ii uv mile s der Beschreis Anmerkung\nbung angesucht\nWohnort des Privilegirten Privilegiums Urkunde in\nward oder nicht\nJahren',
 'Adensamer Joseph, lan Verbesserung an dem Bandwebestuhle unter der 12. October 5 geheim\ndesbefugter Bandfabrikant Benennung Schnell - Gehvorrichtung , &quot; womit 1853\nin Wien ( Schottenfeld) . in ein und derselben Zeit , bei gleichem Kraft\nAufwande noch einmal so viel Waare wie bis\nher erzeugt werden könne .',
 'Adler Carl , Fabrikant Verbesserung in der Erzeugung von Ertrac 22. Februar 1 geheim\nchemisch. Producte in Wien ten aus Knoppern , sowie aus allen gårbe 1853\n( Gumpendorf Nr. 190) . und farbehältigen Stoffen , wodurch mittelst ver\nbindender Anwendung hölzerner , eiserner und\nkupferner Gefäße und Kessel durch offenes\nFeuer oder Dampf diese Extracte mit einem\nErsparnisse

## Second Attempt With Cropping

- We crop the file, then extract the text from the cropped file. Then we use a LLM pipeline to discover the city mentioned. 

In [2]:
# Example usage:
pdf_file_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1853_A.pdf'
output_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/cropped_output.pdf'
extracted_text = crop_pdf_and_extract_text(pdf_file_path, output_path, x1=0, y1=0, x2=180, y2=722)

extracted_text

"Fortlaufende\nVor- und Zuname, Charakter\nBahl\nund\nWohnort des Privilegirten\n1 Adensamer Joseph, lan Ver\ndesbefugter Bandfabrikant Be\nin Wien ( Schottenfeld) . in\nAu\nhe\n2 Adler Carl , Fabrikant Ver\nchemisch. Producte in Wien te\n( Gumpendorf Nr. 190) . un\nbin\nku\nFe\nEr\nmiadar Ar\nste\n3 Alcan Michael , Civil -I n Erfi\ngenieur, und Peter Hypolit ein\nLimet in Paris ( Bevoll Se\nmächtigter Jacob Franz Ab\nHeinrich Hemberger, Fri\nVerwaltungs- Director in bar\nWien) . Tig\n4 Allan Thomas , Ingenieur Erfi\nin Edinburg , durch sei tris\nnen Bevollmächtigten Georg\nMärkl, in Wien (J oseph\nstadt Nr. 65 ) .ad\n5 Altlechner Laurenz , Bür Erfi\nger in Wien (L aimgrube un\nNr. 182) .\nmiadar\n6 Amstötter Mathias , und Erfi\nJohann Schredl , in Wie vo\nner Neustadt Nr . 172. chi\n7 Anthon Ernst Friedrich , Erfi\ntechnisch . Chemiker in Prag, zur\nNr. 70 - IV . höc\nmistop Ve\nsec\nint zu\nKaC\n8 Armengaud Jacob Eugen V\nsen. , Civil Ingenieur in sch\nParis (R ue St. Sebastien zus\n

In [4]:
out = split_text_by_pattern(extracted_text)
out[200:300]

['Knepper Wilhelm , privil .\nBuntpapier Fabrikant in\n=\nWien (W ieden Nr . 348 ) .',
 'Kniely Moriz , wohnhaft\nin Ahgersdorf.',
 'Knieriem Eduard, Sattler\nin Wien ( Leopoldstadt Nr .\n507) .\nKnopp. Siehe Walzl .',
 'König Carl, Chemiker am\nBraunhirschengrund Nr.49,\nnächst Wien .',
 'Kohl August , bürgl . Seiler\nund Martin Johann ,\nbürgerl . Roßhaarsieder in\nWien ( am Laurenzergrund\nNr. 9.)',
 'Kottula Constantin , Lich\nter- und Seifenfabrikant\naus Belgrad in Serbien\n( derzeit in Wien , Schot\ntenfeld Nr . 301 ) .',
 'Krabes Ferdinand , Archi\ntekt und Techniker in Wien\n( Jägerzeile Nr. 403 ) .',
 'Krakowißer Ludwig , ge\nprüfter Apotheker in Wien\n( Stadt Nr. 493 ) .',
 'Kraus Mar , Schneider\nmeister in Pest ( Palatin\nga??e Nr . 3) .',
 'Fortlaufende\nVor- und Zuname, Charakter\nZahl\nund\nWohnort des Privilegirten',
 'Kreuter Franz, Civil - In\ngenieur in Wien .\nKrimmelbein . Siehe\nNr . 259 .',
 'Kreuzer Jos. , Handlungs\nAgent in Wien ( Stadt Nr .\n450) .',
 'Krüge

In [28]:
out2 = request_city_llm(out)
out2

['Sure! The city mentioned in the text is:\n\nBahl',
 'Wien',
 'Wien',
 'Sure! The city mentioned in the text is:\n\nParis',
 'Edinburgh',
 'Altlechner Laurenz',
 'Sure! The city mentioned in the text is:\n\nWien (Vienna)',
 'Prag',
 'Sure! The city mentioned in the text is:\n\nParis',
 'Sure! The city mentioned in the text is:\n\nBerlin',
 'Wien',
 '\nWien',
 'Sure! The city mentioned in the text is:\n\nParis',
 '\nDer Stadtname, der im Text genannt wird, ist:\n\nWien',
 'Asti',
 'Sure! The city mentioned in the text is:\n\nMarien',
 'Sure! The city mentioned in the text is:\n\nPiber',
 'Sure! The city mentioned in the text is:\n\nBologna',
 'Sure! The city mentioned in the text is:\n\nParis',
 'Sure! The city mentioned in the text is:\n\nKrems',
 'London',
 'Sure! The city mentioned in the text is:\n\nTata',
 '\nDie Stadt, die im Text genannt wird, ist:\n\nBerlin',
 'Lettowih',
 'Sure! The city mentioned in the text is:\n\nBahl',
 'Der Städtename ist: Dur.',
 'Stuttgart',
 'Wien',
 '

## A couple of alternatives to check out

- https://ollama.com/marco/em_german_mistral_v01
- https://colab.research.google.com/drive/1IJfJdVwGkfe5MYOqHptystR3FBeEUdGn?usp=sharing
- https://github.com/jphme/EM_German?tab=readme-ov-file
- 